In [1]:
import os
%pwd

'f:\\ml projects\\brain_tumor_classification\\notebooks'

In [2]:
os.chdir("../")
%pwd

'f:\\ml projects\\brain_tumor_classification'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir:Path
    base_model_path:Path
    updated_base_model_path:Path
    params_image_size:list
    params_weights: str
    params_include_top:bool
    params_classes: int
    params_learning_rate:float


In [4]:
from brain_tumor_classification.utils.common import read_yaml,create_directories
from brain_tumor_classification.constants import *

In [5]:
class ConfigurationManager:
    def __init__(self,config_filepath = CONFIG_FILEPATH,params_filepath = PARAMS_FILEPATH) -> None:
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_prepare_base_model_config(self):
        temp_config = self.config.prepare_base_model

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir= Path(temp_config.root_dir),
            base_model_path= Path(temp_config.base_model_path),
            updated_base_model_path= Path(temp_config.updated_base_model_path),
            params_image_size= self.params.IMAGE_SIZE,
            params_weights= self.params.WEIGHTS,
            params_include_top= self.params.INCLUDE_TOP,
            params_classes= self.params.CLASSES,
            params_learning_rate=self.params.LEARNING_RATE,

        )

        return prepare_base_model_config

In [6]:
import tensorflow as tf

In [7]:
class PrepareBaseModel:
    def __init__(self,config:PrepareBaseModelConfig) -> None:
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.applications.VGG16(
            weights=self.config.params_weights,
            include_top = self.config.params_include_top,
            input_shape = self.config.params_image_size
        )
        
        self.model.save(self.config.base_model_path)
    
    @staticmethod
    def _prepare_full_model(model:tf.keras.models,classes:int,learning_rate: float,freeze_all: bool):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        for layer in model.layers:
            print(layer.name,layer.trainable)
        
        full_model = tf.keras.models.Sequential()
        full_model.add(model)
        full_model.add(tf.keras.layers.Flatten())
        full_model.add(tf.keras.layers.Dense(256,activation = 'relu'))
        full_model.add(tf.keras.layers.Dense(units = classes,activation = 'softmax'))


        full_model.compile( 
            optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate),
            loss = tf.keras.losses.CategoricalCrossentropy(),
            metrics = ['accuracy']
            )

        full_model.summary()
        return full_model

    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model = self.model,
            classes = self.config.params_classes,
            learning_rate = self.config.params_learning_rate,
            freeze_all = True
        )
        self.full_model.save(self.config.updated_base_model_path)

In [8]:
from brain_tumor_classification.utils.exception import CustomException
import sys

In [9]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise CustomException(e,sys)

f:\ml projects\brain_tumor_classification\venv\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


input_1 False
block1_conv1 False
block1_conv2 False
block1_pool False
block2_conv1 False
block2_conv2 False
block2_pool False
block3_conv1 False
block3_conv2 False
block3_conv3 False
block3_pool False
block4_conv1 False
block4_conv2 False
block4_conv3 False
block4_pool False
block5_conv1 False
block5_conv2 False
block5_conv3 False
block5_pool False
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 256)               6422784   
                                                                 
 dense_1 (Dense)             (None, 4)                 1028      
                                   